In [10]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options

# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.support.ui import Select, WebDriverWait
# from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup, Comment
import re
import pandas as pd
import os
import numpy as np
import time

In [2]:
def check_done(driver, soup):
    currPage = int(soup.find('li', {'class': 'pagination-info'}).find('strong').text.replace(",", ""))
    finalPage = int(soup.find('li', {'class': 'pagination-info'}).text.split(' ')[-1].replace(",", ""))
    
    if (currPage != finalPage):
        return False, currPage
    else:
        return True, currPage

def click_next(driver):
    _, lastPage = check_done(driver, BeautifulSoup(driver.page_source, 'lxml'))
    requiredPage = lastPage + 1
    
    while True:
        try:
            _, currPage = check_done(driver, BeautifulSoup(driver.page_source, 'lxml'))
            
            if (currPage == requiredPage):
                break
            elif (currPage < requiredPage):
                driver.find_element_by_xpath("//a[contains(text(),'Older Ads')]").click()
            elif (currPage > requiredPage):
                driver.find_element_by_xpath("//a[contains(text(),'Newer Ads')]").click()
                
            time.sleep(1)
        except:
            pass
    
def scrape(links, driver):
    data = pd.DataFrame(columns=['Topic', 'Body', 'in_post_url','Additional Info', 'facebook_url', 'Likes', 'Comments', 'Shares']) 
    
    for link in links:
        driver.get(link)
    
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        
        body = soup1.find('div', {'class':'ads-info'}).text
        additional_info = soup1.find('ul', {'class':'list-custom'}).text
        

        topic = soup1.find(class_='entry-title').get_text()

        newUrl = soup1.find('iframe', {'allow': 'encrypted-media'})['src']
        driver.get(newUrl)

        soup2=BeautifulSoup(driver.page_source, 'lxml')
        
        like = soup2.find('span', {'class':'embeddedLikeButton'}).get_text()
        comment = soup2.find('div', {'title':'Comment'}).get_text()
        share = soup2.find('div', {'title':'Share'}).get_text()
        
        try:
            in_post_link = soup1.find('div', {'class':'ads-info'}).find('a')['href']
        except:
            in_post_link = ""

        data = data.append({'Topic':topic, 'Body': body, 'in_post_url': in_post_link, 'Additional Info': additional_info, 'facebook_url':link, 'Likes':like, 'Comments':comment, 'Shares':share}, ignore_index=True)  
    
    return data

In [3]:
def click_subscribe(driver):
    try:
        driver.find_element_by_name('firstname').send_keys("Daniel")
        driver.find_element_by_name('email').send_keys("daniel@gmail.com")
        driver.find_element_by_xpath("//input[@value='Count me In!']").click()
    except:
        pass

In [11]:
options = Options()
options.headless = True

driver1 = webdriver.Remote(command_executor='http://127.0.0.1:9515', desired_capabilities=DesiredCapabilities.CHROME, options=options)
driver1.set_window_size(1366, 768)

driver2 = webdriver.Remote(command_executor='http://127.0.0.1:9515', desired_capabilities=DesiredCapabilities.CHROME, options=options)
driver2.set_window_size(1366, 768)

In [12]:
driver1.get("https://adespresso.com/ads-examples")

In [13]:
driver1.save_screenshot('a.png')

True

In [8]:
driver1.get("https://adespresso.com/ads-examples")

try:
    driver1.find_element_by_name('firstname').send_keys("Daniel")
    driver1.find_element_by_name('email').send_keys("daniel@gmail.com")
    driver1.find_element_by_xpath("//input[@value='Count me In!']").click()
except:
    pass

In [9]:
initialSoup=BeautifulSoup(driver1.page_source, 'lxml')
is_done, currPage = check_done(driver1, initialSoup)

while is_done == False:
    try:
        print("Scraping page {}".format(str(currPage)))
        initialSoup = BeautifulSoup(driver1.page_source, 'lxml')

        links = initialSoup.find_all('li', {'class': 'sf-item'})
        links = list(set([link.find('a')['href'] for link in links]))

        soup = BeautifulSoup(driver1.page_source, 'lxml')
        df = scrape(links, driver2)

        if currPage == 1:
            df.to_csv('data.csv', index=False)
        else:
            df.to_csv('data.csv', mode='a', header=False, index=False)

        click_next(driver1)
        is_done, currPage = check_done(driver1, initialSoup)
    except Exception as e:
        print("Got an error: {}".format(str(e)))

Scraping page 1
Scraping page 1
Scraping page 1
Scraping page 1
Scraping page 1


KeyboardInterrupt: 